## Deteccion del tablero en la imagen

-----
Se usara el tablero B para esta seccion.

### Representaciones:
<pre>
Se usa una representacion de tablero mediante un array de 64 elementos de 8 bits cada uno. En cada uno de esos elementos se
 almacenara una pieza. Aqui se presentan los encodings para cada una de ellas. 
    * Los primeros 4 bits se usaran para representar el color.
        - 0000 XXXX : Negro
        - 1111 XXXX : Blanco
    * Con los 4 bits restantes se representa la pieza:
         Pieza        Codificacion
        - Peon      | 0000
        - Caballo   | 0001
        - Alfil     | 0010
        - Torre     | 0011
        - Reina     | 0100
        - Rey       | 0101
    * Un casillero vacio se representa mediante: 1111 1111
</pre>
----------------------------------------
### Detalle del algoritmo:
<pre>
El algoritmo consta de 4 partes:
        
    1. Deteccion del tablero: Cuando la foto se captura el tablero puede estar torcido y puesto en cualquier tipo de posicion. Por 
    lo que lo primero es identificarlo, recortarlo, centrarlo y alinearlo.

    2. Luego se reconocen cada una de las casillas individuales. Se determinan sus dimensiones, mediante 2 puntos, la esquina 
    superior izquierda y la esquina inferior izquierda.

    3. Se aplica 2 mascaras. Una de ellas detecta piezas blancas, la otra negras. Si se detecta una pieza se almacena en un array de 
    64 elementos booleanos.

    4. Se compara el array de booleanos con el de piezas y se realizan las debidas actualizaciones.  

A tener en cuenta: 

Para el correcto funcionamiento del algoritmo se debe conocer la posicion inicial de las piezas, ya que en base a estas se realiza 
la deduccion de posiciones despues.
</pre>
--------
#### INVESTIGAR:
* formatos de compresion de la jpeg.
* compresion con perdida y sin perdida.
* investigar el "blur" para aplicar a las imagenes previa a la resta. - blur gausseano 


In [5]:
import cv2
import numpy as np

In [6]:
# Cargo las imagenes del tablero b
b_pos_inicio = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-b/pos-inicial.jpeg")
b_pos_inicio2 = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-b/pos-inicial-2.jpeg")
tablerob_mov1 = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-b/mov-1.jpg")
tablerob_mov2 = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-b/mov-2.jpg")
tablerob_mov3 = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-b/mov-3.jpg")
tablerob_mov4 = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-b/mov-4.jpg")
tablerob_mov5 = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-b/mov-5.jpg")
tablerob_mov6 = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-b/mov-6.jpg")

# Cargo imagenes del tablero sonar
sonar_1 = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/sonar-2-1.jpg")
sonar_2 = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/sonar-2-2.jpg")
sonar_3 = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/sonar-2-3.jpg")
sonar_4 = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/sonar-3-1.jpg")
sonar_5 = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/sonar-3-2.jpg")
sonar_6 = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/sonar-3-4.jpg")
sonar_6 = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/sonar-4.jpg")
sonar_6 = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/sonar-6.jpg")
sonar_6 = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/sonar-7.jpg")
sonar_6 = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/sonar-8.jpg")
sonar_6 = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/sonar-10.jpg")

#Cargo imagenes ideales
ideal = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/tableros-img/workable.jpg")
ideal2 = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/tableros-img/workable2.jpg")

[ WARN:0@0.153] global loadsave.cpp:244 findDecoder imread_('/home/tom/universidad/LIDI/cv-tablero/sonar-2-1.jpg'): can't open/read file: check file path/integrity
[ WARN:0@0.154] global loadsave.cpp:244 findDecoder imread_('/home/tom/universidad/LIDI/cv-tablero/sonar-2-2.jpg'): can't open/read file: check file path/integrity
[ WARN:0@0.154] global loadsave.cpp:244 findDecoder imread_('/home/tom/universidad/LIDI/cv-tablero/sonar-2-3.jpg'): can't open/read file: check file path/integrity
[ WARN:0@0.154] global loadsave.cpp:244 findDecoder imread_('/home/tom/universidad/LIDI/cv-tablero/sonar-3-1.jpg'): can't open/read file: check file path/integrity
[ WARN:0@0.155] global loadsave.cpp:244 findDecoder imread_('/home/tom/universidad/LIDI/cv-tablero/sonar-3-2.jpg'): can't open/read file: check file path/integrity
[ WARN:0@0.155] global loadsave.cpp:244 findDecoder imread_('/home/tom/universidad/LIDI/cv-tablero/sonar-3-4.jpg'): can't open/read file: check file path/integrity
[ WARN:0@0.155] 

In [1]:
# Funciones comunes 

def showImg(img, text='image'):
    cv2.namedWindow(text, cv2.WINDOW_KEEPRATIO)
    cv2.imshow(text, img)
    cv2.resizeWindow(text, 700, 700)
    cv2.waitKey()
    cv2.destroyAllWindows()
    
def mostrarContornos(img,min,max, bool):
    # Aplicar deteccion de bordes utilizando Canny
    edges = cv2.Canny(img, min, max, apertureSize=3)
    if bool:
        showImg(edges, 'Detector de contornos')
    return edges

# 1. Deteccion y recorte del tablero

In [8]:
# Trabajaremos con esta posicion.
showImg(b_pos_inicio, "Posicion inicial")

#### Modelo final

In [10]:
import cv2
import numpy as np

def showImg(img, text='image'):
    cv2.namedWindow(text, cv2.WINDOW_KEEPRATIO)
    cv2.imshow(text, img)
    cv2.resizeWindow(text, 700, 700)
    cv2.waitKey()
    cv2.destroyAllWindows()
    
def mostrarContornos(img,min,max, bool):
    # Aplicar deteccion de bordes utilizando Canny
    edges = cv2.Canny(img, min, max, apertureSize=3)
    if bool:
        showImg(edges, 'Detector de contornos')
    return edges

def rotarRecortarImagen(img, verbose=False):
    # Convertimos la imagen a gris
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    if (verbose): showImg(img_gray, 'Imagen en blanco y negro')

    # Aplicar deteccion de bordes utilizando Canny
    edges = mostrarContornos(img_gray, 150, 300, verbose)

    if (verbose): showImg(edges, 'Contornos detectados')

    # Encontrar las lineas presentes en la imagen utilizando la transformada de Hough
    lines = cv2.HoughLinesP(edges, 1, np.pi/180, threshold=100, minLineLength=100, maxLineGap=10)

    # Calcular el angulo promedio solo de las líneas horizontales detectadas
    angle = np.mean([np.arctan2(line[0][3] - line[0][1], line[0][2] - line[0][0]) for line in lines if abs(line[0][3] - line[0][1]) < abs(line[0][2] - line[0][0])])

    # Rotar la imagen utilizando el ángulo calculado
    (h, w) = img_gray.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle * 180 / np.pi, 1.0)
    rotated = cv2.warpAffine(img, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)

    if (verbose): showImg(rotated, 'Imagen rotada')

    edges = mostrarContornos(rotated, 150, 300, verbose)

    #  Convierto la imagen a hsv
    #hsv_image = cv2.cvtColor(rotated, cv2.COLOR_BGR2HSV)


    #if (verbose): showImg(hsv_image, 'HSV aplicado a la imagen rotada')

    # Defino el rango de colores para poder recortar el tablero, en este caso se usa teniendo en cuenta el fondo de color blanco
    #lower_color = np.array([0, 0, 0])
    #upper_color = np.array([255, 90, 200])

    # Creo una mascara que identifique aproximadamente el color #778078, este es el color del fondo.
    #mask = cv2.inRange(rotated, lower_color, upper_color)

    if (verbose): showImg(edges, 'Mascara, hsv, imagen rotada')

    # Se encuentran los contornos
    contorno,_ = cv2.findContours(edges,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    cont = 0

    booleano = True

    for c in contorno:
        area = cv2.contourArea(c)
        if (area > 10000):
            cont = cont + 1
            x,y,w,h = cv2.boundingRect(c)
            # Recortamos el contorno encontrado.
            if (booleano):
                # rotamos la imagen encontrada
                cropped_image = rotated[(y+10):(y+h-5), (x+10):(x+w-5)].copy()
                if (verbose): showImg(cropped_image, "Imagen recortada")
                booleano = False
            area = cv2.contourArea(c)
    
    if (verbose): showImg(cropped_image, 'Imagen rotada y recortada')

    return cropped_image

def recortarCasillero(image, i, j):
    width = image.shape[1]
    height = image.shape[0]
    start_y = int((height/8)*i)
    end_y = int((height/8)*i+(height/8))
    start_x = int((width/8)*j)
    end_x = int((width/8)*j+(width/8))
    casillero = image[start_y:end_y, start_x:end_x].copy()
    return casillero

def encontrarContornosPieza(image, mask, area_max=6000, area_min=200):
    retorno = image.copy()
    contorno,_ = cv2.findContours(mask,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    
    grid_contours = []   
    for contour in contorno:
        if area_max > cv2.contourArea(contour) > area_min:
            grid_contours.append(contour)

    cv2.drawContours(retorno, grid_contours, -1, (0, 255, 0), 2)
    return retorno, len(grid_contours)

# casilla1 : imagen recortada del casillero 
# verbose : imprime las imagenes de los casilleros con los filtros aplicados y contornos detectados
# white : es un booleano, true para detectar piezas rojas, false para piezas negras
def detectarPieza(casilla1, verbose=False):
    lower_color_black = np.array([0, 0, 0])
    upper_color_black = np.array([90, 90, 90])
    mascara1 = cv2.inRange(casilla1, lower_color_black, upper_color_black).copy()
    img, contorno = encontrarContornosPieza(casilla1,mascara1,6000, 200)
    if verbose:
        showImg(casilla1)
        showImg(img, 'Contornos')
    if contorno > 0:
        if verbose: print('Pieza detectada')
        return 'b'
    else:
        lower_red = np.array([0, 100, 100])
        upper_red = np.array([255, 255, 255])

        mascara1 = cv2.inRange(casilla1, lower_red, upper_red).copy()
        img, contorno = encontrarContornosPieza(casilla1,~mascara1,6000, 100)

        if verbose:
            showImg(casilla1)
            showImg(img, 'Contornos')
        if contorno > 0:
            if verbose: print('Pieza detectada')
            return 'r'
        else:
            if verbose: print('Casillero vacio')
            return '.'    
        




In [10]:
cropped_image = recortarCasillero(image, 4,2)

lower_red = np.array([0, 80, 80])
upper_red = np.array([255, 255, 255])

mascara1 = cv2.inRange(cropped_image, lower_red, upper_red).copy()
img, contorno = encontrarContornosPieza(cropped_image,~mascara1,6000, 100)

print(contorno)
showImg(img)

0


In [11]:
import cv2
import numpy as np

def shadow_removal(image):
  """Removes shadows from an image using OpenCV.

  Args:
    image: The image to remove shadows from.

  Returns:
    The image with shadows removed.
  """

  # Convert the image to the YCbCr color space.
  ycbcr = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)

  # Calculate the shadow mask.
  shadow_mask = np.where((ycbcr[:, :, 0] - ycbcr[:, :, 2]) > 30, 255, 0)

  shadow_mask = shadow_mask.astype("uint8")

  # Remove the shadows from the image.
  shadowless_image = cv2.bitwise_and(image, image, mask=shadow_mask)



  return shadowless_image

image = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/tom.jpg")
# Remove the shadows from the image.
shadowless_image = shadow_removal(image)
# Display the original and shadowless images.
showImg(image, "Original")
showImg(shadowless_image, "Shadowless")


[ WARN:0@1.815] global loadsave.cpp:244 findDecoder imread_('/home/tom/universidad/LIDI/cv-tablero/tom.jpg'): can't open/read file: check file path/integrity


error: OpenCV(4.7.0) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


In [ ]:
def dibujarContornos(img, masked_img, title, bool):
    retorno = img.copy()
    edges = cv2.Canny(masked_img, 200, 500).copy()
    contours, hierarchy = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    chessboard_contours = []
    for contour in contours:
        area = cv2.contourArea(contour)
        if (area > 1000):
            chessboard_contours.append(contour)
    for contour in chessboard_contours:
        cv2.drawContours(retorno , [contour], -1, (0, 255, 0), 2).copy()
    if bool:
        showImg(retorno , title)
    return retorno 

-------
### Deteccion de piezas en casilleros.

In [17]:
import numpy as np
# mascara para casillero blanco pieza blanca
lower_cblanco_pblanco = np.array([100,100,50],np.uint8)
upper_cblanco_pblanco = np.array([170,150,150],np.uint8)

# mascara para casillero negro con pieza negra
lower_cnegro_pnegra = np.array([30,30,30],np.uint8)
upper_cnegro_pnegra = np.array([100,50, 50],np.uint8)

# mascara para casillero negro con pieza blanca
lower_cnegro_pblanca = np.array([0, 0, 0])
upper_cnegro_pblanca = np.array([50, 50, 50])

# mascara para casillero blanco con pieza negra
lower_cblanco_pnegra = np.array([0, 0, 0])
upper_cblanco_pnegra = np.array([50, 50, 50])

lower_mask1 = np.array([100,100,50],np.uint8)
upper_mask1 = np.array([170,150,150],np.uint8)

lower_mask2 = np.array([30,30,30],np.uint8)
upper_mask2 = np.array([100,50, 50],np.uint8)

lower_mask3 = np.array([0, 0, 0])
upper_mask3 = np.array([90,90,90])

In [21]:
import cv2

def detectarPieza(casilla1, verbose=False):
    mascara1 = cv2.inRange(casilla1, lower_mask1, upper_mask1).copy()
    mascara2 = cv2.inRange(casilla1, lower_mask2, upper_mask2).copy()
    mascara3 = cv2.inRange(casilla1, lower_mask3, upper_mask3).copy()
    
    img1, contorno1 = encontrarContornosPieza(casilla1,mascara1)
    img2, contorno2 = encontrarContornosPieza(casilla1,mascara2)
    img3, contorno3 = encontrarContornosPieza(casilla1,mascara3)
    if verbose:
        showImg(casilla1)
        #showImg(img1, 'img1')
        #showImg(img2, 'img2')
        showImg(mascara3, 'mascara3')
        showImg(img3, 'img3')
        print(contorno1,contorno2,contorno3)
    if contorno1 > 0 and contorno2 > 0 and contorno3 > 0:
        if verbose: print('Pieza detectada')
        return '1'
    else:
        if verbose: print('Casillero vacio')
        return '.'        
    

def recortarCasillero(image, i, j):
    start_y = int(100*i)
    end_y = int(97*i+85)
    start_x = int(97*j)
    end_x = int(97*j+85)
    casillero = image[start_y:end_y, start_x:end_x].copy()
    return casillero

img = cv2.imread('casillero.png', cv2.IMREAD_COLOR)
img2 = cv2.imread('casillero2.png', cv2.IMREAD_COLOR)
img3 = cv2.imread('casillero3.png', cv2.IMREAD_COLOR)
img4 = cv2.imread('casillero4.png', cv2.IMREAD_COLOR)
img5 = cv2.imread('casillero5.png', cv2.IMREAD_COLOR)
img6 = cv2.imread('casillero6.png', cv2.IMREAD_COLOR)
img7 = cv2.imread('piezaroja.png', cv2.IMREAD_COLOR)

print(int(np.sum(img)/len(img.ravel())))
print(int(np.sum(img2)/len(img.ravel())))
print(int(np.sum(img3)/len(img.ravel())))
print(int(np.sum(img4)/len(img.ravel())))
print(int(np.sum(img5)/len(img.ravel())))
print(int(np.sum(img6)/len(img.ravel())))
print(int(np.sum(img7)/len(img.ravel())))

detectarPieza(img, True)


90
135
138
169
169
232
1 0 1
Casillero vacio


'.'

In [ ]:
image = rotarRecortarImagen(sonar_5, True) 

#Procesar todo el tablero:
matriz = np.empty((8, 8), dtype=str)
for i in range(8):
    for j in range(8):
        cropped_image = recortarCasillero(image, i,j)
        matriz[i][j] = detectarPieza(cropped_image,False)
print(matriz)     

showImg(image)

[['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '1']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']]


In [ ]:
# TODO: Ver como se puede utilizar la informacion del heatmap, esta interesante ...
hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
h, s, v = cv2.split(hsv_image)
heatmap = cv2.applyColorMap(v, cv2.COLORMAP_HSV)

showImg(heatmap, 'heatmap')

-----------------------
# Tablero B

In [ ]:
 # Carga las imagenes en una variable
a_pos_inicio = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-a/pos-inicio.jpeg")
pos_mid = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-a/pos-mid.jpeg")
pos_mid2 = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-a/pos-mid2.jpeg")
pos_ataque = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-a/pos-ataque.jpeg")
iluminada = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-a/iluminada.jpeg")
pos_inicial_esp = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-a/pos-inicio-esp.jpeg")
pos_mid_esp = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-a/pos-mid-esp.jpeg")
pos_mid2_esp = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-a/pos-mid2-esp.jpeg")
tablero_normal = cv2.imread('/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-ideal/tablero-comun.png')

In [ ]:
# procesamos la imagen 
cv2.imshow('Pos mid', pos_mid)
cv2.waitKey(5000)
cv2.destroyAllWindows()

# Convert the image to grayscale
pos_mid_gray = cv2.cvtColor(pos_mid, cv2.COLOR_BGR2GRAY)

cv2.imshow('Pos mid grayscale', pos_mid_gray)
cv2.waitKey(5000)
cv2.destroyAllWindows()

# Find the chessboard corners
cv2.imshow('tablero normal: ',tablero_normal)
cv2.waitKey(10000)
cv2.destroyAllWindows()

ret, corners = cv2.findChessboardCorners(pos_mid_gray, (9, 9))

print(ret, corners)

# Draw the chessboard corners on the image
if ret:
  cv2.drawChessboardCorners(pos_mid, (9, 9), corners, ret)

# Display the image
cv2.imshow('edges detected', pos_mid) 
cv2.waitKey(5000)
cv2.destroyAllWindows()

False None


In [ ]:
#import cv2
#import numpy as np
#
#def nothing(x):
#    pass
#
## Create a black image, a window
#img = np.zeros((300, 512, 3), np.uint8)
#cv2.namedWindow('image')
#
## create trackbars for color change
#cv2.createTrackbar('R', 'image', 0, 255, nothing)
#cv2.createTrackbar('G', 'image', 0, 255, nothing)
#cv2.createTrackbar('B', 'image', 0, 255, nothing)
## create switch for ON/OFF functionality
#switch = '0 : OFF \n1 : ON'
#cv2.createTrackbar(switch, 'image', 0, 1, nothing)
#while (1):
#    cv2.imshow('image', img)
#    k = cv2.waitKey(1) & 0xFF
#    if k == 27:
#        break
#    # get current positions of four trackbars
#    r = cv2.getTrackbarPos('R', 'image')
#    g = cv2.getTrackbarPos('G', 'image')
#    b = cv2.getTrackbarPos('B', 'image')
#    s = cv2.getTrackbarPos(switch, 'image')
#    if s == 0:
#        img[:] = 0
#    else:
#        img[:] = [b, g, r]
#cv2.destroyAllWindows()
#

# Resta de tableros

ahora viene lo divertido ...    

In [17]:
import numpy as np
import os
import chess, chess.engine

tablero = [['r', 'n', 'b', 'q', 'k', 'b', 'n', 'r'],
           ['p', 'p', 'p', 'p', 'p', 'p', 'p', 'p'],
           ['.', '.', '.', '.', '.', '.', '.', '.'],
           ['.', '.', '.', '.', '.', '.', '.', '.'],
           ['.', '.', '.', '.', '.', '.', '.', '.'],
           ['.', '.', '.', '.', '.', '.', '.', '.'],
           ['P', 'P', 'P', 'P', 'P', 'P', 'P', 'P'],
           ['R', 'N', 'B', 'Q', 'K', 'B', 'N', 'R']]


m0 = np.array([['r', 'r', 'r', 'r', 'r', 'r', 'r', 'r'],
               ['r', 'r', 'r', 'r', 'r', 'r', 'r', 'r'],
               ['.', '.', '.', '.', '.', '.', '.', '.'],
               ['.', '.', '.', '.', '.', '.', '.', '.'],
               ['.', '.', '.', '.', '.', '.', '.', '.'],
               ['.', '.', '.', '.', '.', '.', '.', '.'],
               ['b', 'b', 'b', 'b', 'b', 'b', 'b', 'b'],
               ['b', 'b', 'b', 'b', 'b', 'b', 'b', 'b']])


m1 = [['r', 'r', 'r', 'r', 'r', 'r', 'r', 'r'],
      ['r', 'r', 'r', '.', 'r', 'r', 'r', 'r'],
      ['.', '.', '.', '.', '.', '.', '.', '.'],
      ['.', '.', '.', 'r', '.', '.', '.', '.'],
      ['.', '.', '.', '.', '.', '.', '.', '.'],
      ['.', '.', '.', '.', '.', '.', '.', '.'],
      ['b', 'b', 'b', 'b', 'b', 'b', 'b', 'b'],
      ['b', 'b', 'b', 'b', 'b', 'b', 'b', 'b']]

m2 = [['r', 'r', 'r', 'r', 'r', 'r', 'r', 'r'],
      ['r', 'r', 'r', '.', 'r', 'r', 'r', 'r'],
      ['.', '.', '.', '.', '.', '.', '.', '.'],
      ['.', '.', '.', 'r', '.', '.', '.', '.'],
      ['.', '.', '.', '.', 'b', '.', '.', '.'],
      ['.', '.', '.', '.', '.', '.', '.', '.'],
      ['b', 'b', 'b', 'b', '.', 'b', 'b', 'b'],
      ['b', 'b', 'b', 'b', 'b', 'b', 'b', 'b']]

m0_n = np.array([[2, 2, 2, 2, 2, 2, 2, 2],
                 [2, 2, 2, 2, 2, 2, 2, 2],
                 [0, 0, 0, 0, 0, 0, 0, 0],
                 [0, 0, 0, 0, 0, 0, 0, 0],
                 [0, 0, 0, 0, 0, 0, 0, 0],
                 [0, 0, 0, 0, 0, 0, 0, 0],
                 [1, 1, 1, 1, 1, 1, 1, 1],
                 [1, 1, 1, 1, 1, 1, 1, 1]])

m1_mov_blanco = np.array([[2, 2, 2, 2, 2, 2, 2, 2],
                          [2, 2, 2, 0, 2, 2, 2, 2],
                          [0, 0, 0, 0, 0, 0, 0, 0],
                          [0, 0, 0, 2, 0, 0, 0, 0],
                          [0, 0, 0, 0, 0, 0, 0, 0],
                          [0, 0, 0, 0, 0, 0, 0, 0],
                          [1, 1, 1, 1, 1, 1, 1, 1],
                          [1, 1, 1, 1, 1, 1, 1, 1]])

m2_mov_negro = np.array([[2, 2, 2, 2, 2, 2, 2, 2],
                         [2, 2, 2, 0, 2, 2, 2, 2],
                         [0, 0, 0, 0, 0, 0, 0, 0],
                         [0, 0, 0, 2, 0, 0, 0, 0],
                         [0, 0, 0, 0, 1, 0, 0, 0],
                         [0, 0, 0, 0, 0, 0, 0, 0],
                         [1, 1, 1, 1, 0, 1, 1, 1],
                         [1, 1, 1, 1, 1, 1, 1, 1]])

m3_blancocomenegro = np.array([[2, 2, 2, 2, 2, 2, 2, 2],
                               [2, 2, 2, 0, 2, 2, 2, 2],
                               [0, 0, 0, 0, 0, 0, 0, 0],
                               [0, 0, 0, 0, 0, 0, 0, 0],
                               [0, 0, 0, 0, 2, 0, 0, 0],
                               [0, 0, 0, 0, 0, 0, 0, 0],
                               [1, 1, 1, 1, 0, 1, 1, 1],
                               [1, 1, 1, 1, 1, 1, 1, 1]])

m4_negrocomeblanco = np.array([[2, 2, 2, 2, 2, 2, 2, 2],
                               [2, 2, 2, 0, 2, 2, 2, 2],
                               [0, 0, 0, 0, 0, 0, 0, 0],
                               [0, 0, 0, 1, 0, 0, 0, 0],
                               [0, 0, 0, 0, 0, 0, 0, 0],
                               [0, 0, 0, 0, 0, 0, 0, 0],
                               [1, 1, 1, 1, 0, 1, 1, 1],
                               [1, 1, 1, 1, 1, 1, 1, 1]])

# print(res)
# print(puntos)

matriz_mov_tipos = np.array([[1, -1],  # mov negro 0
                            [2, -1],  # pieza blanca come pieza negra 1
                            [1, 1],   # pieza negra come pieza blanca 2
                            [2, -2]])  # mov blanco 3


def matriz_a_fen(tablero):
    fen = ""
    for fila in tablero:
        vacios = 0
        for celda in fila:
            if celda == ".":
                vacios += 1
            else:
                if vacios > 0:
                    fen += str(vacios)
                    vacios = 0
                fen += celda
        if vacios > 0:
            fen += str(vacios)
        fen += "/"
    fen = fen[:-1]  # Eliminar la barra diagonal final
    return fen


def tienen_mismos_numeros(array1, array2):
    conjunto1 = set(array1)
    conjunto2 = set(array2)
    return conjunto1 == conjunto2


def determinar_puntos(mov0, mov1, cont_peon_capturas):

    res = mov0 - mov1

    puntos = np.zeros((2, 3), dtype=int)
    cont = 0

    # puntos =  [indice][res, x, y]
    for i in range(8):
        for j in range(8):
            if (res[i][j] != 0):
                puntos[cont][0] = res[i][j]
                puntos[cont][1] = i
                puntos[cont][2] = j
                cont = cont + 1

    diferencias = np.array([puntos[0][0], puntos[1][0]])

    caso = -1

    for i in range(4):
        if (tienen_mismos_numeros(diferencias, matriz_mov_tipos[i])):
            caso = i
            break

    origen = np.zeros((2), dtype=int)
    destino = np.zeros((2), dtype=int)

    if (caso == 0):
        for i in range(2):
            if (puntos[i][0] == 1):
                origen[0] = int(puntos[i][1])
                origen[1] = int(puntos[i][2])
            else:
                destino[0] = int(puntos[i][1])
                destino[1] = int(puntos[i][2])
    elif (caso == 1):
        cont_peon_capturas = cont_peon_capturas + 1 
        for i in range(2):
            if (puntos[i][0] == 2):
                origen[0] = puntos[i][1]
                origen[1] = puntos[i][2]
            else:
                destino[0] = puntos[i][1]
                destino[1] = puntos[i][2]
    elif (caso == 2):
        cont_peon_capturas = cont_peon_capturas + 1 
        for i in range(2):
            if (mov1[puntos[i][1]][puntos[i][2]] == 0):
                origen[0] = puntos[i][1]
                origen[1] = puntos[i][2]
            else:
                destino[0] = puntos[i][1]
                destino[1] = puntos[i][2]
    elif (caso == 3):
        for i in range(2):
            if (puntos[i][0] == 2):
                origen[0] = puntos[i][1]
                origen[1] = puntos[i][2]
            else:
                destino[0] = puntos[i][1]
                destino[1] = puntos[i][2]
    return origen, destino, cont_peon_capturas


def mejor_movimiento(fen):
    engine = chess.engine.SimpleEngine.popen_uci(
        os.getcwd() + "/Stockfish/src/stockfish")
    tablero = chess.Board(fen)
    result = engine.play(tablero, chess.engine.Limit(time=2.0))
    print("Movimiento a realizar --> " + str(result.move) + "\n")
    engine.quit()
    return (result.move)

def fen_to_chessboard(fen_string):
  rows = fen_string.split("/")
  chessboard = []
  for row in rows:
    chessboard.append([piece for piece in row])

  # Handle castling rights.

  castling_rights = fen_string[-2:]
  for i, piece in enumerate(chessboard[0]):
    if piece == "K":
      if castling_rights[0] == "K":
        chessboard[0][i + 1] = "k"  # White king side castle
      elif castling_rights[1] == "K":
        chessboard[0][i - 2] = "k"  # White queen side castle
    elif piece == "k":
      if castling_rights[2] == "k":
        chessboard[7][i + 1] = "K"  # Black king side castle
      elif castling_rights[3] == "k":
        chessboard[7][i - 2] = "K"  # Black queen side castle

  return chessboard



cont_jugadas = 0
cont_peon_capturas = 0
debug = False

origen, destino, status = determinar_puntos(m0_n, m1_mov_blanco, cont_peon_capturas)

tablero[destino[0]][destino[1]] = tablero[origen[0]][origen[1]]
tablero[origen[0]][origen[1]] = '.'


print("Resultado: \n Tablero leido exitosamente!\n")                
for i in range(8):
    print(tablero[i])   
cont_jugadas = cont_jugadas + 1


fen = matriz_a_fen(tablero) + str(" b KQkq - " + str(cont_peon_capturas) + " " + str(cont_jugadas))
#print(fen)
# Ingreso el string fen al chess engine
move = mejor_movimiento(fen)
# Pusheo el movimiento al tablero
chessboard = chess.Board(fen)
chessboard.push(move)
print(chessboard.fen())

board = fen_to_chessboard(str(chessboard.fen()))

print(board[i])


Resultado: 
 Tablero leido exitosamente!

['r', 'n', 'b', 'q', 'k', 'b', 'n', 'r']
['p', 'p', 'p', '.', 'p', 'p', 'p', 'p']
['.', '.', '.', '.', '.', '.', '.', '.']
['.', '.', '.', 'p', '.', '.', '.', '.']
['.', '.', '.', '.', '.', '.', '.', '.']
['.', '.', '.', '.', '.', '.', '.', '.']
['P', 'P', 'P', 'P', 'P', 'P', 'P', 'P']
['R', 'N', 'B', 'Q', 'K', 'B', 'N', 'R']
Movimiento a realizar --> e7e5

rnbqkbnr/ppp2ppp/8/3pp3/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 2


IndexError: string index out of range

In [10]:
origen, destino = determinarPuntos(m0_n.copy(), m1_mov_blanco.copy())


tablero[destino[0]][destino[1]] = tablero[origen[0]][origen[1]]
tablero[origen[0]][origen[1]] = '.'

for i in range(8):
    print(tablero[i])

[[ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0 -1  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  1  0  0  0]
 [ 0  0  0  0  0  0  0  0]]
[[-1  4  4]
 [ 1  6  4]]
0
[6 4] [4 4]


In [11]:
origen, destino = determinarPuntos(m2_mov_negro, m3_blancocomenegro)

print(origen, destino)

[[ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  2  0  0  0  0]
 [ 0  0  0  0 -1  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]]
[[ 2  3  3]
 [-1  4  4]]
1
[3 3] [4 4]


In [15]:
origen, destino = determinarPuntos(m2_mov_negro, m4_negrocomeblanco)

print(origen, destino)

[[0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0]
 [0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]]
[[1 3 3]
 [1 4 4]]
2
[4 4] [3 3]


In [2]:
def matriz_a_fen(tablero):
    fen = ""
    for fila in tablero:
        vacios = 0
        for celda in fila:
            if celda == ".":
                vacios += 1
            else:
                if vacios > 0:
                    fen += str(vacios)
                    vacios = 0
                fen += celda
        if vacios > 0:
            fen += str(vacios)
        fen += "/"
    fen = fen[:-1]  # Eliminar la barra diagonal final
    return fen

# Representación matricial del tablero (8x8)
tablero_matriz = [
    ["r", "n", "b", "q", "k", "b", "n", "r"],
    ["p", "p", "p", "p", "p", "p", "p", "p"],
    [".", ".", ".", ".", ".", ".", ".", "."],
    [".", ".", ".", ".", ".", ".", ".", "."],
    [".", ".", ".", ".", ".", ".", ".", "."],
    [".", ".", ".", ".", ".", ".", ".", "."],
    ["P", "P", "P", "P", "P", "P", "P", "P"],
    ["R", "N", "B", "Q", "K", "B", "N", "R"]
]

fen_resultante = matriz_a_fen(tablero_matriz) + str(" w KQkq - 0 0")
print("FEN resultante:", fen_resultante)

FEN resultante: rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 0


In [9]:
import chess

# Create a chessboard
board = chess.Board("rnbqkbnr/pppppppp/8/8/3P4/8/PPP1PPPP/RNBQKBNR b KQkq - 0 1")

# Apply a movement: e2 to e4
movement = "e2e4" 
move = chess.Move.from_uci(movement)
board.push(move)

# Print the updated chessboard
print(type(board))

<class 'chess.Board'>


In [2]:
apply_move(chess_board, "Nc3")

# Imprimir el tablero después del movimiento
for row in chess_board:
    print(' '.join(row))

KeyError: 'c'

In [6]:
import chess, chess.engine, os

engine = chess.engine.SimpleEngine.popen_uci(os.getcwd() + "/Stockfish/src/stockfish")
tablero = chess.Board("rnbqkbnr/pppppppp/8/8/3P4/8/PPP1PPPP/RNBQKBNR b KQkq - 0 1")
result = engine.play(tablero, chess.engine.Limit(time=2.0))  
print("Movimiento a realizar --> " + str(result.move) + "\n")
print(tablero)
print(str(tablero.__str__()))
tablero_str = tablero.__str__()
engine.quit()

Movimiento a realizar --> d7d5

r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . P . . . .
. . . . . . . .
P P P . P P P P
R N B Q K B N R
r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . P . . . .
. . . . . . . .
P P P . P P P P
R N B Q K B N R


In [15]:
matriz = np.zeros((8, 8), dtype=int)
for i in range(0, len(tablero_str), 2):
    if (tablero_str[i].isalpha()):
        if (tablero_str[i].isupper()):
            matriz[i//16][cont] = 2
        if (tablero_str[i].islower()):
            matriz[i//16][cont] = 1
    matriz[i//16][cont]
    cont = cont + 1 
    if (cont == 8): cont = 0
print(matriz)

[[1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 2 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [2 2 2 0 2 2 2 2]
 [2 2 2 2 2 2 2 2]]


In [12]:
import numpy as np
matriz = np.zeros((8, 8), dtype=int)
cont = 0
for i in range(0, len(tablero_str), 2):
    print(str(i//16), str(cont))
    cont = cont + 1 
    if (cont == 8): cont = 0


0 0
0 1
0 2
0 3
0 4
0 5
0 6
0 7
1 0
1 1
1 2
1 3
1 4
1 5
1 6
1 7
2 0
2 1
2 2
2 3
2 4
2 5
2 6
2 7
3 0
3 1
3 2
3 3
3 4
3 5
3 6
3 7
4 0
4 1
4 2
4 3
4 4
4 5
4 6
4 7
5 0
5 1
5 2
5 3
5 4
5 5
5 6
5 7
6 0
6 1
6 2
6 3
6 4
6 5
6 6
6 7
7 0
7 1
7 2
7 3
7 4
7 5
7 6
7 7


In [18]:
fen = "rnbqkbnr/pp3ppp/2p1p3/3p4/3P4/2N2N2/PPP2PPP/R1BQKB1R w KQkq - 1 6"
parts = fen.split('-')
number_list = parts[1].split()
numbers = [int(number) for number in number_list]
print(numbers[0])
print(numbers[1]) 

1
6


In [46]:
def mejor_movimiento(fen):
    engine = chess.engine.SimpleEngine.popen_uci(os.getcwd() + "/Stockfish/src/stockfish")
    tablero = chess.Board(fen)
    result = engine.play(tablero, chess.engine.Limit(time=2.0))  
    print("Movimiento a realizar --> " + str(result.move) + "\n")
    engine.quit()
    return result

def extraer_contadores_string_fen(fen):
    parts = fen.split('-')
    number_list = parts[1].split()
    numbers = [int(number) for number in number_list]
    return numbers[0],numbers[1] 

def tablero_a_matriz_numerica(tablero):
    matriz = np.zeros((8, 8), dtype=int)
    matriz2 = np.zeros((8, 8), dtype=str)
    cont = 0
    for i in range(0, len(tablero.__str__()), 2):
        if (tablero.__str__()[i].isalpha()):
            if (tablero.__str__()[i].isupper()):
                matriz[i//16][cont] = 2
            elif (tablero.__str__()[i].islower()):
                matriz[i//16][cont] = 1
        matriz2[i//16][cont] = tablero.__str__()[i]
        cont = cont + 1 
        if (cont == 8): cont = 0
    return matriz, matriz, matriz2.copy()

fen = "rnbqkbnr/pppppppp/8/8/8/7P/PPPPPPP1/RNBQKBNR b KQkq - 0 1"
chessboard = chess.Board(fen)     
# Ingreso el string fen al chess engine
                    
move = mejor_movimiento(fen)

chessboard.push_san(str(move.move))

# Pusheo el movimiento al tablero 
                    
print(chessboard)
                    
matriz_numerica_t0, matriz_numerica_t1, tablero = tablero_a_matriz_numerica(chessboard)
cont_peon_capturas, cont_jugadas = extraer_contadores_string_fen(chessboard.fen())

for i in range(8):
    print(tablero[i])

Movimiento a realizar --> e7e5

r n b q k b n r
p p p p . p p p
. . . . . . . .
. . . . p . . .
. . . . . . . .
. . . . . . . P
P P P P P P P .
R N B Q K B N R
['r' 'n' 'b' 'q' 'k' 'b' 'n' 'r']
['p' 'p' 'p' 'p' '.' 'p' 'p' 'p']
['.' '.' '.' '.' '.' '.' '.' '.']
['.' '.' '.' '.' 'p' '.' '.' '.']
['.' '.' '.' '.' '.' '.' '.' '.']
['.' '.' '.' '.' '.' '.' '.' 'P']
['P' 'P' 'P' 'P' 'P' 'P' 'P' '.']
['R' 'N' 'B' 'Q' 'K' 'B' 'N' 'R']


In [5]:
import chess

fen = "8/5k2/3p4/1p1Pp2p/pP2Pp1P/P4P1K/8/8 b - - 99 50"

chessboard = chess.Board(fen)
# Ingreso el string fen al chess engine
chessboard.is_checkmate()
# Pusheo el movimiento al tablero 

False

In [8]:
import pyglet
pyglet.resource.media('jaquemate.ogg').play()


In [7]:
from checkpoint import Partida

part = Partida()
part.setParams('hola','hola1','hola2','hola3')


0

hola3
